## Setup & Configuration

In [1]:
#Go to "ADAPET" Directory
%cd ADAPET

/Users/david/Desktop/fewshot_emotions/ADAPET


In [ ]:
!pip install -r requirements.txt

#### Environmental Variables

In [51]:
%env ADAPET_ROOT= /content/drive/MyDrive/ADAPET
%env PYTHONPATH=$ADAPET_ROOT:$PYTHONPATH
%env PYTHON_EXEC=python

env: ADAPET_ROOT=/content/drive/MyDrive/ADAPET
env: PYTHONPATH=$ADAPET_ROOT:$PYTHONPATH
env: PYTHON_EXEC=python


In [ ]:
# !rm -r exp_out

## Dataset

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold

In [45]:
df = pd.read_csv("train.txt", delimiter=';', header=None, names=['sentence','label'])
df, _ = train_test_split(df, train_size=0.05,stratify=df['label'],random_state=20)
# from sklearn.preprocessing import LabelEncoder
# labelencoder = LabelEncoder()
# df['label_enc'] = labelencoder.fit_transform(df['label'])
df.rename(columns={'sentence':'TEXT1'},inplace=True)
df.rename(columns={'label':'LBL'},inplace=True)

In [46]:
df['LBL'].unique()


array(['anger', 'joy', 'sadness', 'fear', 'love', 'surprise'],
      dtype=object)

In [47]:
train_df , eval_df = train_test_split(df, train_size=0.6,stratify=df['LBL'],random_state=20)
eval_df , test_df = train_test_split(eval_df, train_size=0.6,stratify=eval_df['LBL'],random_state=20)

In [48]:
train_df.to_json('data/my_task/train.jsonl',lines=True,orient='records')
test_df.to_json('data/my_task/test.jsonl',lines=True,orient='records')
eval_df.to_json('data/my_task/eval.jsonl',lines=True,orient='records')

In [49]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2fabe14d-7a70-9249-9aad-523adbb133aa)


## PROMPT SEARCH
You should run evaluations to see which prompt gives best output. Skipped in this notebook due to computation requirements.

In [29]:
# skf = StratifiedKFold(n_splits=5)

In [30]:
# for train_index, test_index in skf.split(train_df['TEXT1'], train_df['LBL']):
    #evaluate here

In [31]:
# !sh bin/test.sh exp_out/generic/albert-xxlarge-v2/v1f1/
# saves = ['v1f1'....]
# import pandas as pd
# from sklearn.metrics import classification_report
# for save in saves:
#     predictions = pd.read_json(f"exp_out/generic/albert-xxlarge-v2/{save}/test.json",lines=True)
#     truth = pd.read_json("data/my_task/test.jsonl",lines=True)
#     print(classification_report(list(truth['LBL']),list(predictions['label'])))

## TEST

In [53]:
!python cli.py --data_dir data/my_task --pattern '[TEXT1] The emotion epxressed is [LBL]'  --dict_verbalizer '{"anger":"anger", "joy":"joy", "sadness":"sadness", "fear":"fear", "love":"love", "surprise":"surprise"}' --num_batches 50  --dir_names 'emotion'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
Downloading: 100% 710/710 [00:00<00:00, 837kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 7.75MB/s]
Downloading: 100% 1.31M/1.31M [00:00<00:00, 11.1MB/s]
Downloading: 100% 893M/893M [00:24<00:00, 36.5MB/s]


In [55]:
!sh bin/test.sh exp_out/generic/albert-xxlarge-v2/emotion/
predictions = pd.read_json(f"exp_out/generic/albert-xxlarge-v2/emotion/test.json",lines=True)
truth = pd.read_json("data/my_task/test.jsonl",lines=True)
from sklearn.metrics import classification_report
print(classification_report(list(truth['LBL']),list(predictions['label'])))

+ exp_dir=exp_out/generic/albert-xxlarge-v2/emotion/
+ python -m src.test -e exp_out/generic/albert-xxlarge-v2/emotion/
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
              precision    recall  f1-score   support

       anger       0.82      0.53      0.64        17
        fear       0.67      0.62      0.65        16
         joy       0.61      0.88      0.72        43
        love       0.67      0.20      0.31        10
     sadness       0.67      0.65      0.66        37
    surprise       0.00      0.00      0.00         5

    accuracy                           0.65       128
   macro avg       0.57      0.48      0.50       128
weighted avg       0.64      0.65      0.62       128



The performance is not that great but mainly because I haven't done any validation to find best prompt.